In [ ]:
from qubecalib.neopulseexp import Sequence, Slot

with Sequence() as seq:
    Slot(100e-9)
    Slot(200e-9)
    Slot(300e-9)

seq._tree.place_slots()
seq, seq._tree._nodes_items

In [ ]:
from qubecalib.neopulseexp import Sequence, SubSequence, Slot, Series, Flushleft, Flushright

with Sequence() as seq:
    with Series():
        Slot(1e-9)
        Slot(2e-9)

seq._tree.place_slots()
seq._tree._nodes_items, seq._get_tree()

In [ ]:
from qubecalib.neopulseexp import Sequence, SubSequence, Slot, Series, Flushleft, Flushright

with Sequence() as seq:
    with Flushleft():
        Slot(1e-9)
        Slot(2e-9)

seq._tree.place_slots()
seq._tree._nodes_items, seq._get_tree()

In [ ]:
from qubecalib.neopulseexp import Sequence, SubSequence, Slot, Series, Flushleft, Flushright

with Sequence() as seq:
    with Flushright():
        Slot(1e-9)
        Slot(2e-9)

seq._tree.place_slots()
seq._tree._nodes_items, seq._get_tree(), seq._tree._tree._cost

In [ ]:
from qubecalib.neopulseexp import Sequence, SubSequence, Slot, Series, Flushleft, Flushright

with Sequence() as seq:
    with SubSequence():
        Slot(9e-9)
        with Flushright():
            with Series():
                Slot(1e-9)
                Slot(2e-9)
            with Series():
                Slot(3e-9)
                Slot(4e-9)
        with Flushleft():
            with Series():
                Slot(5e-9)
                Slot(6e-9)
            with Series():
                Slot(7e-9)
                Slot(8e-9)

print(seq._tree._nodes_items)
seq._tree.place_slots()
seq._tree._nodes_items, seq._get_tree()#, seq._tree._tree._cost


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from qubecalib.neopulseexp import Sequence, SubSequence, Slot, Series, Flushleft, Flushright, Branch
from qubecalib.neopulseexp import Sequence, SubSequence, Slot, Series, Flushleft, Flushright

%matplotlib inline

with Sequence() as seq:
    with SubSequence():
        Slot(1e-9)
        Slot(2e-9)

seq._tree.place_slots()

fig = plt.figure()
ax = fig.gca()
for i, _ in seq._tree._nodes_items.items():
    if isinstance(_, Branch):
        print(f"{_.__class__.__name__}, {_.begin}, {_.duration}")
        ax.add_patch(patches.Rectangle(xy=(_.begin*1e9, i), width=_.duration*1e9, height=1, color='green'))
    if isinstance(_, Slot):
        ax.add_patch(patches.Rectangle(xy=(_.begin*1e9, i), width=_.duration*1e9, height=1, color='blue'))
ax.set_xlim(0,5)
ax.set_ylim(0,10)
plt.show()

seq._tree._nodes_items, seq._get_tree()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from qubecalib.neopulseexp import Sequence, SubSequence, Slot, Series, Flushleft, Flushright, Branch
from qubecalib.neopulseexp import Sequence, SubSequence, Slot, Series, Flushleft, Flushright, RaisedCosFlatTop

%matplotlib inline

with Sequence() as seq:
    with SubSequence():
        Slot(9e-9)
        with Flushright():
            with Series():
                Slot(1e-9)
                Slot(2e-9)
            with Series():
                Slot(3e-9)
                Slot(4e-9)
        Slot(10e-9)
        with Flushleft():
            with Series():
                Slot(5e-9)
                Slot(6e-9)
            with Series():
                Slot(7e-9)
                Slot(8e-9)

seq._tree.place_slots()

fig = plt.figure()
ax = fig.gca()
for i, _ in seq._tree._nodes_items.items():
    if isinstance(_, Branch):
        # print(f"{_.__class__.__name__}, {_.begin}, {_.duration}")
        ax.add_patch(patches.Rectangle(xy=(_.begin*1e9, i), width=_.duration*1e9, height=1, color='green'))
    if isinstance(_, Slot):
        ax.add_patch(patches.Rectangle(xy=(_.begin*1e9, i), width=_.duration*1e9, height=1, color='blue'))
ax.set_xlim(0,50)
ax.set_ylim(0,30)
plt.show()

seq._tree._nodes_items, seq._get_tree()#, seq._tree._tree._cost


In [ ]:
import math, cmath
class Test:
    def __init__(self):
        self.amplitude = 1.
        self.phase = 0.
    def __rmul__(self, other: complex):
        p = self.phase / 180. * math.pi
        c = self.amplitude * (math.cos(p) + 1j * math.sin(p))
        r = other * c
        self.amplitude = math.sqrt((r * r.conjugate()).real)
        self.phase = math.atan2(r.imag, r.real)/math.pi * 180
        return self
    def __repr__(self) -> str:
        return f"{self.__class__.__name__}(amplitude={self.amplitude}, phase={self.phase})"


c1 = Test()
0.5 * cmath.exp(1j*math.pi/2) * Test()

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import qubecalib.neopulseexp as pls
from qubecalib.neopulseexp import Sequence, SubSequence, Slot, Series, Flushleft, Flushright, RaisedCosFlatTop, TargetHolder, Blank, Range, Branch, SubSequenceBranch
from qubecalib.neopulseexp import Modifier, RaisedCosFlatTop
from typing import Optional
%matplotlib inline

class THSlot(Slot, TargetHolder):
    def __init__(self, duration: Optional[float] = None) -> None:
        super().__init__(duration)
        TargetHolder.__init__(self)

with Sequence() as seq:
    with SubSequence():
        Modifier().set_target("CQ1")
        with Flushright():
            with Series():
                RaisedCosFlatTop(50e-9).set_target("CQ1")
                pls.Rectangle(50e-9).set_target("CQ1")
                pls.Arbit(50e-9).set_target("CQ1")
                with Flushleft():
                    with Series():
                        Blank(50e-9)
                        RaisedCosFlatTop(100e-9).set_target("RQ1GEN")
                    Range(200e-9).set_target("RQ1CAP")
    with SubSequence():
        Modifier().set_target("CQ1")
        # with Flushright():
        with Series():
            pls.Arbit(50e-9).set_target("CQ1")
            RaisedCosFlatTop(50e-9).set_target("CQ1")
            pls.Rectangle(50e-9).set_target("CQ1")
            with Flushleft():
                with Series():
                    Blank(50e-9)
                    RaisedCosFlatTop(100e-9).set_target("RQ1GEN")
                Range(200e-9).set_target("RQ1CAP")
    # Sequence のトップレベルテスト
    with Flushright():
        Slot(5e-9)
        Slot(30e-9)
    with Flushleft():
        Slot(15e-9)
        Slot(40e-9)
    Slot(100e-9)
    # SubSequence の入れ子テスト
    with SubSequence():
        # Flushright の入れ子テスト
        with Flushright():
            with Series():
                Slot(5e-9)
                Slot(10e-9)
            with Series():
                Slot(25e-9)
                Slot(30e-9)
        # Flushleft の入れ子テスト
        with Flushleft():
            with Series():
                Slot(15e-9)
                Slot(20e-9)
            with Series():
                Slot(35e-9)
                Slot(40e-9)
    # Series の入れ子テスト
    with Series():
        # Flushright の入れ子テスト
        with Flushright():
            Slot(5e-9)
            Slot(30e-9)
        # Flushleft の入れ子テスト
        with Flushleft():
            Slot(15e-9)
            Slot(40e-9)

seq._tree.place_slots()

fig = plt.figure()
ax = fig.gca()
for i, _ in seq._tree._nodes_items.items():
    if isinstance(_, SubSequenceBranch):
        ax.add_patch(patches.Rectangle(xy=(_.begin*1e9, i), width=_.duration*1e9, height=1, color='lightgreen'))
        continue
    if isinstance(_, Branch):
        ax.add_patch(patches.Rectangle(xy=(_.begin*1e9, i), width=_.duration*1e9, height=1, color='green'))
        continue
    if isinstance(_, Blank):
        ax.add_patch(patches.Rectangle(xy=(_.begin*1e9, i), width=_.duration*1e9, height=1, color='lightgray'))
        continue
    if isinstance(_, Range):
        ax.add_patch(patches.Rectangle(xy=(_.begin*1e9, i), width=_.duration*1e9, height=1, color='orange'))
        continue
    if isinstance(_, Modifier):
        ax.plot(_.begin*1e9, i, '*', color='red')
        continue
    if isinstance(_, Slot):
        ax.add_patch(patches.Rectangle(xy=(_.begin*1e9, i), width=_.duration*1e9, height=1, color='blue'))
        continue
ax.set_xlim(0,1200)
ax.set_ylim(0,100)
plt.show()

targets_items = seq._get_group_items_by_target()

fig = plt.figure()
ax = fig.gca()
target_counter = 0
for targets, items in targets_items.items():
    for node, _items in items.items():
        for _ in _items:
            if isinstance(_, Branch):
                ax.add_patch(patches.Rectangle(xy=(_.begin*1e9, target_counter), width=_.duration*1e9, height=1, color='green'))
                continue
            if isinstance(_, Blank):
                ax.add_patch(patches.Rectangle(xy=(_.begin*1e9, target_counter), width=_.duration*1e9, height=1, color='lightgray'))
                continue
            if isinstance(_, Range):
                ax.add_patch(patches.Rectangle(xy=(_.begin*1e9, target_counter), width=_.duration*1e9, height=1, color='orange'))
                continue
            if isinstance(_, Modifier):
                ax.plot(_.begin*1e9, target_counter, '*', color='red')
                continue
            if isinstance(_, Slot):
                ax.add_patch(patches.Rectangle(xy=(_.begin*1e9, target_counter), width=_.duration*1e9, height=1, color='blue'))
                continue
    target_counter += 1
ax.set_xlim(0,900)
ax.set_ylim(0,len(targets_items))
plt.show()

targets_items, seq._tree._nodes_items, seq._tree._tree._tree

[<qubecalib.neopulseexp.SequenceTree object at 0x7f380ac378b0>, <qubecalib.neopulseexp.SequenceTree object at 0x7f380abed130>, <qubecalib.neopulseexp.SequenceTree object at 0x7f380abefdf0>, <qubecalib.neopulseexp.SequenceTree object at 0x7f380abef910>, <qubecalib.neopulseexp.SequenceTree object at 0x7f380abe9af0>]
{1: SubSequenceBranch(duration=None, begin=None, next_node=5, root_node=6), 2: Dummy(begin=None), 3: Modifier(begin=None), 4: FlushrightBranch(duration=None, begin=None, next_node=8, root_node=9), 5: Dummy(begin=None), 6: Padding(duration=0, begin=None), 7: SeriesBranch(duration=None, begin=None, next_node=11, root_node=12), 8: Dummy(begin=None), 9: RaisedCosFlatTop(duration=5e-08, begin=None), 10: Rectangle(duration=5e-08, begin=None), 11: Arbit(duration=5e-08, begin=None), 12: FlushleftBranch(duration=None, begin=None, next_node=16, root_node=17), 13: Dummy(begin=None), 14: SeriesBranch(duration=None, begin=None, next_node=18, root_node=19), 15: Dummy(begin=None), 18: Range

KeyError: 162

In [ ]:
for targets, items in targets_items.items():
    for node, _items in items.items():
        for _ in _items:


In [ ]:
{item._root_node: item for item in seq._tree._nodes_items.values() if isinstance(item, SubSequenceBranch)}

In [ ]:
[seq._tree._nodes_items[_] for _ in seq._tree.breadth_first_search(2) if isinstance(seq._tree._nodes_items[_], TargetHolder)]

In [ ]:
{{"a": 1, "duration": 2.5e-7}: "item"}

In [ ]:
{
    "CQ1": {0: ["Slot"], 1: ["Slot", "Slot"]},
}
0: SubSequenceBranch
1: SubSequenceBranch